In [ ]:
!pip install tensorflow pandas

In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten,Dropout

In [3]:
train_df = pd.read_csv('/kaggle/input/brackish-tf/Brackish 960x540.tensorflow/train/_annotations.csv')
test_df = pd.read_csv('/kaggle/input/brackish-tf/Brackish 960x540.tensorflow/test/_annotations.csv')
valid_df = pd.read_csv('/kaggle/input/brackish-tf/Brackish 960x540.tensorflow/valid/_annotations.csv')


In [4]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


In [5]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='/kaggle/input/brackish-tf/Brackish 960x540.tensorflow/train',
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

Found 28518 validated image filenames belonging to 6 classes.


In [6]:
valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid_df,
    directory='/kaggle/input/brackish-tf/Brackish 960x540.tensorflow/valid',
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

Found 3581 validated image filenames belonging to 6 classes.


In [7]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='/kaggle/input/brackish-tf/Brackish 960x540.tensorflow/test',
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)


Found 3466 validated image filenames belonging to 6 classes.


In [8]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = Flatten()(x)
x = Dense(460, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(58, activation='relu')(x)
x = Dense(188, activation='relu')(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False


In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
# 
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n // valid_generator.batch_size,
    epochs=10
)


Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1718023507.191623     405 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1718023507.212267     405 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


198/445 ━━━━━━━━━━━━━━━━━━━━ 1:57 476ms/step - accuracy: 0.4907 - loss: 4.3280

W0000 00:00:1718023601.142330     405 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


445/445 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step - accuracy: 0.5721 - loss: 2.7771

W0000 00:00:1718023674.518073     407 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


445/445 ━━━━━━━━━━━━━━━━━━━━ 247s 411ms/step - accuracy: 0.5723 - loss: 2.7737 - val_accuracy: 0.7369 - val_loss: 0.6123
Epoch 2/10
  1/445 ━━━━━━━━━━━━━━━━━━━━ 1:50 248ms/step - accuracy: 0.7188 - loss: 0.6317

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
W0000 00:00:1718023744.429092     408 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


445/445 ━━━━━━━━━━━━━━━━━━━━ 55s 122ms/step - accuracy: 0.7188 - loss: 0.6317 - val_accuracy: 0.7869 - val_loss: 0.6248
Epoch 3/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 144s 318ms/step - accuracy: 0.7553 - loss: 0.6083 - val_accuracy: 0.7702 - val_loss: 0.5313
Epoch 4/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 1s 583us/step - accuracy: 0.7500 - loss: 0.6175 - val_accuracy: 0.8033 - val_loss: 0.4934
Epoch 5/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 142s 315ms/step - accuracy: 0.7604 - loss: 0.5842 - val_accuracy: 0.7830 - val_loss: 0.5170
Epoch 6/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 1s 555us/step - accuracy: 0.7969 - loss: 0.5211 - val_accuracy: 0.8197 - val_loss: 0.5358
Epoch 7/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 143s 317ms/step - accuracy: 0.7646 - loss: 0.5477 - val_accuracy: 0.7869 - val_loss: 0.5011
Epoch 8/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 0s 546us/step - accuracy: 0.8125 - loss: 0.4941 - val_accuracy: 0.8033 - val_loss: 0.5077
Epoch 9/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 143s 317ms/step - accuracy: 0.7684 - loss: 0.5398 - va

In [11]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.n // test_generator.batch_size)
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)


54/54 ━━━━━━━━━━━━━━━━━━━━ 17s 303ms/step - accuracy: 0.7988 - loss: 0.4541
Test loss: 0.4737350046634674
Test accuracy: 0.7879050970077515


In [20]:
import matplotlib.pyplot as plt

def plot_history(history, title):
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{title} Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{title} Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.show()

In [25]:
# Plot history for VGG16
plot_history(history,'VGG16')

NameError: name 'history' is not defined

In [26]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 460)            │    11,540,940 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 460)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 58)             │        26,738 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 188)            │        11,092 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │         1,13

 Total params: 49,454,402 (188.65 MB)

 Trainable params: 11,579,904 (44.17 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

 Optimizer params: 23,159,810 (88.35 MB)

In [27]:
# Save the VGG16 model
model.save('Vgg16_Brackish(78.79%).h5')